In [1]:
!pip install pulp

In [2]:
import pandas as pd
import pulp

In [3]:
factories = pd.read_csv('factory_variables.csv', index_col=['Month','Factory'], names=['Month', 'Factory', 'Max_Capacity', 'Min_Capacity', 'Variable_Costs', 'Fixed_Costs'])
factories

Max_Capacity  Min_Capacity  Variable_Costs  Fixed_Costs
Month Factory                                                         
1     A              100000         20000              10          500
      B               50000         20000               5          600
2     A              110000         20000              11          500
      B               55000         20000               4          600
3     A              120000         20000              12          500
      B               60000         20000               3          600
4     A              145000         20000               9          500
      B              100000         20000               5          600
5     A              160000         20000               8          500
      B                   0             0               0            0
6     A              140000         20000               8          500
      B               70000         20000               6          600
7     A              155000         20000               5          500
      B               60000         20000               4          600
8     A              200000         20000               7          500
      B              100000         20000               6          600
9     A              210000         20000               9          500
      B              100000         20000               8          600
10    A              197000         20000              10          500
      B              100000         20000              11          600
11    A               80000         20000               8          500
      B              120000         20000              10          600
12    A              150000         20000               8          500
      B              150000         20000              12          600

In [4]:
demand = pd.read_csv('monthly_demand.csv', index_col=['Month'], names=['Month', 'Demand'])
demand

,Demand
Month,
1,120000
2,100000
3,130000
4,130000
5,140000
6,130000
7,150000
8,170000
9,200000


In [5]:
production = pulp.LpVariable.dicts("production",
                                   ((month, factory) for month, factory in factories.index),
                                   lowBound=0,
                                   cat='Integer')

In [6]:
factory_status = pulp.LpVariable.dicts("factory_status",
                                     ((month, factory) for month, factory in factories.index),
                                     cat='Binary')

In [7]:
model = pulp.LpProblem("Cost_minimising_scheduling_problem", pulp.LpMinimize)

In [8]:
model += pulp.lpSum(
    [production[month, factory] * factories.loc[(month, factory), 'Variable_Costs'] for month, factory in factories.index]
    + [factory_status[month, factory] * factories.loc[(month, factory), 'Fixed_Costs'] for month, factory in factories.index]
)

In [9]:
# Production in any month must be equal to demand
months = demand.index
for month in months:
    model += production[(month, 'A')] + production[(month, 'B')] == demand.loc[month, 'Demand']

In [10]:
# Production in any month must be between minimum and maximum capacity, or zero.
for month, factory in factories.index:
    min_production = factories.loc[(month, factory), 'Min_Capacity']
    max_production = factories.loc[(month, factory), 'Max_Capacity']
    model += production[(month, factory)] >= min_production * factory_status[month, factory]
    model += production[(month, factory)] <= max_production * factory_status[month, factory]

In [11]:
# Factory B is off in May
model += factory_status[5, 'B'] == 0
model += production[5, 'B'] == 0

In [12]:
model.solve()
pulp.LpStatus[model.status]

'Optimal'

In [13]:
output = []
for month, factory in production:
    var_output = {
        'Month': month,
        'Factory': factory,
        'Production': production[(month, factory)].varValue,
        'Factory Status': factory_status[(month, factory)].varValue
    }
    output.append(var_output)
output_df = pd.DataFrame.from_records(output).sort_values(['Month', 'Factory'])
output_df.set_index(['Month', 'Factory'], inplace=True)
output_df

Production  Factory Status
Month Factory                            
1     A           70000.0             1.0
      B           50000.0             1.0
2     A           45000.0             1.0
      B           55000.0             1.0
3     A           70000.0             1.0
      B           60000.0             1.0
4     A           30000.0             1.0
      B          100000.0             1.0
5     A          140000.0             1.0
      B               0.0             0.0
6     A           60000.0             1.0
      B           70000.0             1.0
7     A           90000.0             1.0
      B           60000.0             1.0
8     A           70000.0             1.0
      B          100000.0             1.0
9     A          100000.0             1.0
      B          100000.0             1.0
10    A          190000.0             1.0
      B               0.0             0.0
11    A           80000.0             1.0
      B           60000.0             1.0
12    A          100000.0             1.0
      B               0.0             0.0

In [ ]:
# Print our objective function value (Total Costs)
print(pulp.value(model.objective))